In [2]:
import argparse
import json
import numpy as np

- 데이터가 사람과 영화선호도 구성되어 있다면
- 두 사람을 서로 비교하는 방법을 알아야한다. - 유사성 점수
- 유사성 점수 : 두 포인트가 얼마나 유사한지
- 유클리드점수, 피어슨 점수
- 유클리드점수가 0과 1사이가 되도록 , 두 개체간의 유클리드 거리가 크면 유클리드 점수는 낮다
- 피어슨 점수 : -1 ~ 1 사이의 값을 갖는다

In [ ]:
# 입력 인수를 처리할 파서 - 사용자 두명과, 유사성 점수를 계산하는데 사용할 점수 유형을 얻음
def build_arg_parser():
  parser = argparse.ArgumentParser(description='Compute similarity score')
  parser.add_argument('--user1',dest='user1',required=True, help='First user')
  parser.add_argument('--user2',dest='user2',required=True, help='Second user')
  parser.add_argument('--score-type',dest='score_type',required=True,
                      choices=['Euclidean','Pearson'], help='Similarity metric to be used')
  return parser

In [ ]:
# 콘솔에서 python script.py --user1 ABC --user2 BBB --score-type Euclidean

In [5]:
# 입력한 사용자간의 유클리드 점수를 계산하는 함수
# 사용자가 데이터셋에 없으면 에러
def euclidean_score(dataset, user1,user2):
  if user1 not in dataset:
    raise TypeError(f"Cannot find {user1} in the dataset")
  if user2 not in dataset:
    raise TypeError(f"Cannot find {user2} in the dataset")
  # 두 사용자가 평가한 영화를 추적하는 변수를 정의
  # 사용자1과 사용자2가 평가한 영화
  common_movies = {}
  # 두 사용자가 평가한 영화를 추출
  for item in dataset[user1]:
    if item in dataset[user2]:
      common_movies[item] = 1
  # 두 사용자간의 공통된 영화가 없으면 유사성 점수는 0
  if len(common_movies) ==0:
    return 0
  # 두 평점간의 제곱차이를 계사니하고, 이를 사용해 유클리드 점수를 계산
  squared_diff = []
  for item in dataset[user1]:
    if item in dataset[user2]:
      squared_diff.append(np.square(dataset[user1][item] - dataset[user2][item]))
  return 1 / (1+ np.sqrt(np.sum(squared_diff)))  # 1 / (1+sqrt(평점  차이 제곱의 합))

# 피어슨 상관관계 점수 계산
def pearson_score(dataset,user1,user2):
  if user1 not in dataset:
    raise TypeError(f"Cannot find {user1} in the dataset")
  if user2 not in dataset:
    raise TypeError(f"Cannot find {user2} in the dataset")
  # 사용자1과 사용자2가 평가한 영화
  common_movies = {}
  # 두 사용자가 평가한 영화를 추출
  for item in dataset[user1]:
    if item in dataset[user2]:
      common_movies[item] = 1
  # 두 사용자간의 공통된 영화가 없으면 상관관계를 못구함
  if len(common_movies) ==0:
    return 0

  # 두 사용자가 모두 평가한 모든 영화의 평점의 합을 계산
  user1_sum = np.sum(  dataset[user1][item] for item in common_movies  )
  user2_sum = np.sum(  dataset[user2][item] for item in common_movies  )

  # 두 사용자가 모두 평가한 모든 영화의 평점제곱의 합을 계산
  user1_squared_sum = np.sum(  np.square( dataset[user1][item]) for item in common_movies  )
  user2_squared_sum = np.sum(  np.square( dataset[user2][item]) for item in common_movies  )

  # 두 사용자가 모두 평가한 모든 영화의 평점의 곱의 합
  sum_of_products = np.sum( np.sum( dataset[user1][item]*dataset[user2][item] ) for item in common_movies  )

  num_rations = len(common_movies)

  #피어슨 상관관계 점수 계산
  Sxy =  sum_of_products - (user1_sum*user2_sum / num_rations)  # 공분산
  Sxx = user1_squared_sum - np.square(user1_sum) / num_rations  # user1의 분산
  Syy = user2_squared_sum - np.square(user2_sum) / num_rations  # user2의 분산

  # 편차가 없으면 점수는 0
  if Sxx * Syy == 0:
    return 0

  return Sxy / np.sqrt(Sxx * Syy)

In [9]:
with open('ratings.json', 'r') as f:
  data = json.loads(f.read())

In [12]:
# 유사성 점수를 계산
score_type = 'Euclidean'
if score_type == 'Euclidean':
  print("Euclidean score")
  print(euclidean_score(data,'David Smith','Bill Duffy'))
else:
  print("Pearson score")
  print(pearson_score(data,'David Smith','Bill Duffy'))

Euclidean score
0.585786437626905


In [13]:
print("Pearson score")
print(pearson_score(data,'David Smith','Bill Duffy'))

Pearson score
0.9909924304103233


<ipython-input-5-b83b92821c50>:42: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  user1_sum = np.sum(  dataset[user1][item] for item in common_movies  )
<ipython-input-5-b83b92821c50>:43: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  user2_sum = np.sum(  dataset[user2][item] for item in common_movies  )
<ipython-input-5-b83b92821c50>:46: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  user1_squared_sum = np.sum(  np.square( dataset[user1][item]) for item in common_movies  )
<ipython-input-5-b83b92821c50>:47: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will 

협업필터링을 사용해서 유사한 사용자 찾기
  - 협업필터링 : 새로운 객체를 찾아내기 위해 데이터 세트 내 객체 간에 패턴을 찾는 프로세스
  - 데이터  세트에서 유사한 사용자를 살펴봄으로써 추천을 제공

In [42]:
# compute_scores.py 파일을 로드한다(git 14주차에 있음)
from compute_scores import pearson_score
# import sys
# print(sys.modules.get("compute_scores") )
# importlib.reload("compute_scores")

In [47]:
# 데이터세트에서 입력한 사용자와 유사한 사용자를 찾는 함수 - 데이터세트에 사용자 없으면 오류
def find_similar_users(dataset, user,num_users):
  if user not in dataset:
    raise TypeError(f'Cannot find {user} in the dataset')
  #입력한 사용자와 테이터 세트내의 모든 사용간의 피어슨 점수 계산
  scores = np.array(    [  [x, pearson_score(dataset,user,x)]    for x in dataset if x != user   ]        )
  # 점수를 내림차순으로 정렬

  scores_sorted =  np.argsort(scores[:,1])[::-1]
  # 상위 num_users개의 점수 추출
  top_users = scores_sorted[:num_users]
  return scores[top_users]

In [51]:
# 평점데이터를 로드
with open('ratings.json', 'r') as f:
  data = json.loads(f.read())
# 기존 평점에 있는 유저중에 이 유저와 비슷한 유저를 찾을수 있다.
user = "Bill Duffy"
similar_user =  find_similar_users(data,'Brenda Peterson',5)
similar_user

array([['Julie Hammel', '0.9962709627734359'],
       ['Chris Duncan', '-1.0'],
       ['Bill Duffy', '-0.9466130952336697'],
       ['Samuel Miller', '-0.9299279982188647'],
       ['Clarissa Jackson', '-0.7335065881732874']], dtype='<U32')

영화 추천 시스템 구축

In [64]:
# 입력 사용자를 위한 영화 추천
def get_recommendations(dataset,input_user):
  if input_user not in dataset:
    raise TypeError(f'Cannot not {input_user} in the dataset')
  # 점수를 추적할 변수
  overall_scores = {}
  similarity_scores = {}
  # 유사성 점수 계산
  for user in   [x for x in dataset if x != input_user]:
    similarity_score = pearson_score(dataset,input_user,user)
    if similarity_score <=0:  # 유사성 점수가 0보다 큰 대상만
      continue
    # 현재  사용자가 평가했지만 입력한 유저가 평가하지 않은 영화 목록
    filtered_list = [x for x in dataset[user] if x not in dataset[input_user] or dataset[input_user][x] == 0]
    # 필터링한 영화 목록 내 각 항목에 대해, 유사성 점수를 기반으로 가중치가 부여된 평점을 추적, 유사성 점수도 추적
    for item in filtered_list:
      overall_scores.update({item:dataset[user][item]*similarity_score})
      similarity_scores.update({item:similarity_score})
    if len(overall_scores) == 0:
      return "no recommendations possible"
    # 가중치 점수에 기반한 점수를 정규화 한다
    movie_scores =  np.array( [  [score/similarity_scores[item], item ]  for item,score in  overall_scores.items()]    )
    # 점수를 정렬하고 추천 영화를 추출한다
    #내림차순
    movie_scores =  movie_scores[np.argsort(movie_scores[:,0])[::-1]]
    # 영화 추천 추출
    movie_recommendations =  [ movie for _, movie in movie_scores ]
    return movie_recommendations



In [65]:
# 영화평점 데이터 로드
# 평점데이터를 로드
with open('ratings.json', 'r') as f:
  data = json.loads(f.read())
movie = get_recommendations(data,'Chris Duncan')

In [67]:
for idx, data in enumerate(movie):
  print(f"{idx+1}. {data}")

1. Goodfellas
2. Scarface
3. Vertigo
